# IIJA All Projects 

In [1]:
import numpy as np
import pandas as pd
from siuba import *

from shared_utils import geography_utils
from dla_utils import _dla_utils

from calitp import to_snakecase

import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)


## Read Data

In [3]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [4]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/CopyofFMIS_Projects_Universe_IIJA_Reporting_4.xls", 
                           # sheet_name='FMIS 5 Projects  ', header=[3]
                           sheet_name='IIJA',
                           # sheet_name='FMIS 5 Projects  ',
                           ))

In [5]:
proj.drop(columns =['unnamed:_0', 'unnamed:_13', 'unnamed:_14'], axis=1, inplace=True)

In [6]:
proj.sample(5)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value
231,2022-06-28,YS30,HIGHWAY SAFETY IMP PROG IIJA,5213058,0417000332L,VARIOUS LOCATION IN SUNNYVALE PROVIDE ADVANCE DILEMMA ZONE DETECTION.,85.00,Cong Dist 17,15.00,Preliminary Engineering,"11,728.10",L5213MTC
1005,2022-09-13,Y001,NATIONAL HIGHWAY PERF IIJA,X059084,1218000034S,"CONST BRIDGES, RW, SW, JPCP, HMA PVMT, DRAINAGE AND ELECT SYSTEMS. ORANGE COUNTY IN IRVINE, SANTA ANA, AND TUSTIN ON ROUTE 5 AT NEWPORT AVENUE UNDERC",59.00,Cong Dist 45,4.00,4R - No Added Capacity,"1,670,189.18",S SCAG
75,2022-06-13,Y001,NATIONAL HIGHWAY PERF IIJA,5008201,1022000248L,MULTIPLE BRIDGE LOCATIONS (29C0093L; 29C0093R; 29C0237; 20C0242; 29C0400) BRIDGE PREVENTIVE MAINTENANCE: EROSION REPAIRS; METHACRYLATE DECK SEALING;,77.00,Cong Dist 9,15.00,Preliminary Engineering,"270,920.00",L5008SJCOG
1247,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,22.00,Rail/Hwy Crossing,"18,900.00",S MTC
903,2022-05-02,Y001,NATIONAL HIGHWAY PERF IIJA,P395348,0900000030S,IN INYO COUNTY NEAR OLANCHA AND CARTAGO FROM 1.4 MILES SOUTH OF LOS ANGELES AQUEDUCT BRIDGE TO 0.2 MILE SOUTH OF ASH CREEK BRIDGE. CONSTRUCT 4 LANE E,27.00,Cong Dist 8,42.00,Training,"17,800.00",S NON-MPO


In [7]:
proj = proj.dropna(how='all') 

In [8]:
proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].fillna(value='None')

In [9]:
proj.head(1)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value
0,2022-03-03,Y230,STBG-URBANIZED >200K IIJA,5002174,0314000222L,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),67.00,Cong Dist 6,4.00,4R - No Added Capacity,"2,841,397.00",L5002SACOG


In [10]:
len(proj)

1241

In [11]:
## agency identifier column doesnt always have locode

## Function Work

In [12]:
# (proj[proj['summary_recipient_defined_text_field_1_value'].str.startswith('L')]).sample()

In [13]:
# (proj[proj['summary_recipient_defined_text_field_1_value'].str.startswith('L')]).sample()

In [14]:
#seeing if the space is the way to filter
# ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")])>>count(_.summary_recipient_defined_text_field_1_value))

In [15]:
#check to make sure there are numbers
# ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")])>>count(_.summary_recipient_defined_text_field_1_value)).iloc[1:20]

In [16]:
# locode_proj = ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))
# locode_proj = locode_proj>>filter(_.summary_recipient_defined_text_field_1_value!='None')

In [17]:
# len(locode_proj)

In [18]:
# locode_proj = (utils.add_name_from_locode(locode_proj, 'summary_recipient_defined_text_field_1_value'))

In [19]:
# no_locode = ((proj[proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))

In [20]:
# no_entry = proj>>filter(_['summary_recipient_defined_text_field_1_value']=='None')
    
#     #concat no locodes and those with no entry
# no_locode = pd.concat([no_locode, no_entry])

In [21]:
# no_locode.sample()

In [22]:
# no_locode = no_locode.rename(columns = {"recipient_name":"implementing_agency", "county_description":"county_name"})

In [23]:
# len(no_locode)

In [24]:
# pd.concat([locode_proj, no_locode])

## Function for adding Implemeting Agency Information

In [25]:
# function moved to utils

In [83]:
df = utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [84]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
106,2022-09-12,YS30,HIGHWAY SAFETY IMP PROG IIJA,P025033,0517000185S,SAN BENITO COUNTY NEAR HOLLISTER ON ROUTE 25 FROM 0.4 MILE EAST TO 0.3 MILE WEST OF ROUTE 156 AND ON ROUTE 156 FROM 0.4 MILE SOUTH TO 0.3 MILE NORTH,69.00,Cong Dist 20,17.00,Construction Engineering,"2,165,000.00",S AMBAG,NaN,California,5.00,San Benito County,NaN,NaN


In [85]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 0 to 671
Data columns (total 18 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fmis_transaction_date                         1254 non-null   datetime64[ns]
 1   program_code                                  1254 non-null   object        
 2   program_code_description                      1254 non-null   object        
 3   project_number                                1254 non-null   object        
 4   recipient_project_number                      1253 non-null   object        
 5   project_title                                 1254 non-null   object        
 6   county_code                                   1254 non-null   float64       
 7   congressional_district                        1254 non-null   object        
 8   improvement_type                              1254 non-null   float64

In [86]:
df = (utils.add_description(df, 'project_title'))

In [87]:
df>>filter(_.project_type=="Project")

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_method,project_type,other
0,2022-03-03,Y230,STBG-URBANIZED >200K IIJA,5002174,0314000222L,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),67.00,Cong Dist 6,4.00,4R - No Added Capacity,"2,841,397.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project,Other Projects
1,2022-03-03,Y230,STBG-URBANIZED >200K IIJA,5002174,0314000222L,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),67.00,Cong Dist 6,17.00,Construction Engineering,"255,927.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project,Other Projects
2,2022-08-26,Y400,CONGESTION MITIGATION IIJA,5002178,0315000001L,"SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO",67.00,Cong Dist 6,17.00,Construction Engineering,"50,000.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project,Other Projects
3,2022-08-26,Y400,CONGESTION MITIGATION IIJA,5002178,0315000001L,"SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO",67.00,Cong Dist 6,28.00,Facilities for Pedestrians and Bicycles,"508,614.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project,Other Projects
4,2022-07-15,Y400,CONGESTION MITIGATION IIJA,5002189,0316000158L,ALONG UNUSED RAIL CORRIDOR NEAR FREEPORT BLVD FROM SUTTERVILLE ROAD TO SOUTH OF MEADOWVIEW/POCKET ROAD (4.5 MILES) RAILS TO TRAILS PROJECT ; NEW CLAS,67.00,Cong Dist 6,28.00,Facilities for Pedestrians and Bicycles,"4,359,000.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,New,Project,Other Projects
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,2022-06-16,Y001,NATIONAL HIGHWAY PERF IIJA,4053001,0716000031S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,37.00,Cong Dist 29,44.00,Other,"2,475,335.29",S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN,,Project,Other Projects
663,2022-06-16,Y001,NATIONAL HIGHWAY PERF IIJA,4053001,0716000031S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,37.00,Cong Dist 30,17.00,Construction Engineering,"185,535.45",S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN,,Project,Other Projects
664,2022-06-16,Y001,NATIONAL HIGHWAY PERF IIJA,4053001,0716000031S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,37.00,Cong Dist 30,42.00,Training,353.05,S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN,,Project,Other Projects
665,2022-06-16,Y001,NATIONAL HIGHWAY PERF IIJA,4053001,0716000031S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,37.00,Cong Dist 30,44.00,Other,"448,864.71",S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN,,Project,Other Projects


In [88]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_method,project_type,other
148,2022-09-07,Y003,PROJ TO REDUCE PM 2.5 EMI IIJA,5085057,1021000097L,MERCED NORTH TRANSIT HUB DESIGN AND CONSTRUCTION OF PARKING AREA AND TRANSIT HUB IN BELLEVUE RANCH DEVELOPMENT,47.00,Cong Dist 16,1.00,New Construction Roadway,"152,837.00",L5085MCAG,"5,085.00",Merced,10.00,Merced County,Merced County Association of Governments,Merced County Association Of Goverments,Construct,Signage,Other Projects


In [89]:
def add_new_title(df, first_col_method, second_col_type, third_col_name, alt_col_name):
    """
    Function to add new title. 
    Expected output example: "New Bike Lane in Eureka"
    """
    def return_name(df):
        if (df[third_col_name] == "California"):
            return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_col_name])
    
        elif (df[third_col_name] != "California"):
            return (df[first_col_method] + " " + df[second_col_type] + " in " + df[third_col_name])

        return df

    df['project_name_new'] = df.apply(return_name, axis = 1)
    
    return df

In [92]:
df = add_new_title(df, "project_method", "project_type", "implementing_agency", "county_name")

In [94]:
df.head()


,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_method,project_type,other,project_name_new
0,2022-03-03,Y230,STBG-URBANIZED >200K IIJA,5002174,0314000222L,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),67.00,Cong Dist 6,4.00,4R - No Added Capacity,"2,841,397.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project,Other Projects,Project in Sacramento
1,2022-03-03,Y230,STBG-URBANIZED >200K IIJA,5002174,0314000222L,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),67.00,Cong Dist 6,17.00,Construction Engineering,"255,927.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project,Other Projects,Project in Sacramento
2,2022-08-26,Y400,CONGESTION MITIGATION IIJA,5002178,0315000001L,"SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO",67.00,Cong Dist 6,17.00,Construction Engineering,"50,000.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project,Other Projects,Project in Sacramento
3,2022-08-26,Y400,CONGESTION MITIGATION IIJA,5002178,0315000001L,"SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO",67.00,Cong Dist 6,28.00,Facilities for Pedestrians and Bicycles,"508,614.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project,Other Projects,Project in Sacramento
4,2022-07-15,Y400,CONGESTION MITIGATION IIJA,5002189,0316000158L,ALONG UNUSED RAIL CORRIDOR NEAR FREEPORT BLVD FROM SUTTERVILLE ROAD TO SOUTH OF MEADOWVIEW/POCKET ROAD (4.5 MILES) RAILS TO TRAILS PROJECT ; NEW CLAS,67.00,Cong Dist 6,28.00,Facilities for Pedestrians and Bicycles,"4,359,000.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,New,Project,Other Projects,New Project in Sacramento


### Export progress

In [100]:
#test_df = utils.title_column_names(df)

In [99]:
#test_df.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_wip_all.csv")